In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

from delta import *

spark = (
    SparkSession
    .builder.appName('Delta')
    .master('local[*]')
    .config('spark.dynamicAllocation.enabled', 'false')
    .config('spark.jars.packages', 'io.delta:delta-core_2.12:2.4.0')
    .config('spark.sql.extensions', 'io.delta.sql.DeltaSparkSessionExtension')
    .config('spark.sql.catalog.spark_catalog',
            'org.apache.spark.sql.delta.catalog.DeltaCatalog')

    .getOrCreate()            
)

sc = spark.sparkContext

23/11/19 00:14:06 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.100.7 instead (on interface wlp3s0)
23/11/19 00:14:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/radi/.pyenv/versions/3.12.0/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/radi/.ivy2/cache
The jars for the packages stored in: /home/radi/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-82889983-7042-4d2d-a465-aec0901500c2;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 281ms :: artifacts dl 11ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   

In [2]:
import pyspark
pyspark.__version__

'3.4.0'

In [3]:
yello_taxi_schema = StructType([
    StructField('VendorID', IntegerType(), True),
    StructField('tpep_pickup_datetime', TimestampType(), True),
    StructField('tpep_dropoff_datetime', TimestampType(), True),
    StructField('passenger_count', DoubleType(), True),
    StructField('trip_distance', DoubleType(), True),
    StructField('RatecodeID', DoubleType(), True),
    StructField('store_and_fwd_flag', StringType(), True),
    StructField('PULocationID', IntegerType(), True),
    StructField('DOLocationID', IntegerType(), True),
    StructField('payment_type', IntegerType(), True),
    StructField('fare_amount', DoubleType(), True),
    StructField('extra', DoubleType(), True),
    StructField('mta_tax', DoubleType(), True),
    StructField('tip_amount', DoubleType(), True),
    StructField('tolls_amount', DoubleType(), True),
    StructField('improvement_surcharge', DoubleType(), True),
    StructField('total_amount', DoubleType(), True),
    StructField('congestion_surcharge', DoubleType(), True),
    StructField('airport_fee', DoubleType(), True),
])

In [4]:
yellow_taxi_df = (
    spark
        .read
        .option('header', 'true')
        .schema(yello_taxi_schema)
        .csv('../data/taxi/YellowTaxis_202210.csv')
)

In [5]:
yellow_taxi_df.show(10, truncate=False)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|1       |2022-10-01 03:03:41 |2022-10-01 03:18:39  |1.0            |1.7          |1.0       |N                 |249         |107         |1           |9.5        |3.0  |0.5    |2.65     

In [6]:
spark.sql("""
    CREATE DATABASE IF NOT EXISTS TaxiDB
          
          """)

DataFrame[]

In [7]:
(yellow_taxi_df
    .write
    .mode('overwrite')
    .partitionBy('VendorID')
    .format('parquet')
    .option('path', '/home/radi/Projects/DataLab/data/output/YelloTaxis.parquet')
    .saveAsTable('TaxiDB.YellowTaxisParquet')
)

23/11/19 00:14:54 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/11/19 00:14:57 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [13]:
30/43

0.6976744186046512

In [9]:


(yellow_taxi_df
    .write
    .mode('overwrite')
    .partitionBy('VendorID')
    .format('delta')
    .option('path', '/home/radi/Projects/DataLab/data/output/YelloTaxis.delta')
    .saveAsTable('TaxiDB.YellowTaxis')
)

23/11/19 00:16:16 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/11/19 00:16:16 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/11/19 00:16:17 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/11/19 00:16:17 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/11/19 00:16:17 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/11/19 00:16:17 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/11/19 00:16:17 WARN MemoryManager: Total allocation exceeds 95.00% 

In [11]:
spark.sql("""
SELECT * FROM TaxiDB.YellowTaxis

""").show(10, truncate=False)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|2       |2022-10-03 18:17:20 |2022-10-03 18:32:06  |1.0            |1.83         |1.0       |N                 |230         |143         |1           |11.0       |0.0  |0.5    |2.86     

In [36]:
spark.sql("""
DESCRIBE HISTORY TaxisDB.YellowTaxis
""").show(truncate=False)

+-------+-----------------------+------+--------+---------------------------------+----------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+----------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp              |userId|userName|operation                        |operationParameters                                                                     |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                      |userMetadata|engineInfo                         |
+-------+-----------------------+------+--------+---------------------------------+----------------------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+-----------------------------------------------

In [37]:
spark.sql(
"""
CREATE TABLE TaxisDB.YellowTaxisNew
(
VendorId    INT     COMMENT 'Vendor providing ride',

PickupTime  TIMESTAMP,
DropTime    TIMESTAMP,

PickupLocationId    INT NOT NULL,
DropLocationID  INT,

PassengerCount  DOUBLE,
TripDistance    DOUBLE,

RateCodeId  DOUBLE,
StoreAndFwdFlag STRING,
PaymentType INT,

FareAmount  DOUBLE,
Extra       DOUBLE,
MtaTax      DOUBLE,
TipAmount   DOUBLE,
TollsAmount DOUBLE,
ImprovementSurcharge DOUBLE,
TotalAmount DOUBLE,
CongestionSurcharge DOUBLE,
AirportFee DOUBLE

)    

USING DELTA

LOCATION "/home/raddy/projects/DataLab/spark-tutorials/data/output/YelloTaxisNew.delta"

PARTITIONED BY (VendorId)

COMMENT 'This table stores ride information for Yellow Taxis'

"""
)

AnalysisException: [TABLE_OR_VIEW_ALREADY_EXISTS] Cannot create table or view `TaxisDB`.`YellowTaxisNew` because it already exists.
Choose a different name, drop or replace the existing object, or add the IF NOT EXISTS clause to tolerate pre-existing objects.

In [38]:
spark.sql(
"""
DESCRIBE TABLE EXTENDED TaxisDB.YellowTaxisNew
"""
).show(50, truncate=False)

+----------------------------+---------------------------------------------------------------------------------+---------------------+
|col_name                    |data_type                                                                        |comment              |
+----------------------------+---------------------------------------------------------------------------------+---------------------+
|VendorId                    |int                                                                              |Vendor providing ride|
|PickupTime                  |timestamp                                                                        |null                 |
|DropTime                    |timestamp                                                                        |null                 |
|PickupLocationId            |int                                                                              |null                 |
|DropLocationID              |int                      

In [42]:
spark.sql("""
        SELECT INPUT_FILE_NAME()
          ,VendorID
          ,PULocationID
          ,passenger_count

          FROM TaxisDB.YellowTaxis
          WHERE VendorId=3

          """).show(10, truncate=False)

+-----------------+--------+------------+---------------+
|input_file_name()|VendorID|PULocationID|passenger_count|
+-----------------+--------+------------+---------------+
+-----------------+--------+------------+---------------+

